# [STARTER] Udaplay Project

## Part 01 - Offline RAG

In this part of the project, you'll build your VectorDB using Chroma.

The data is inside folder `project/starter/games`. Each file will become a document in the collection you'll create.
Example.:
```json
{
  "Name": "Gran Turismo",
  "Platform": "PlayStation 1",
  "Genre": "Racing",
  "Publisher": "Sony Computer Entertainment",
  "Description": "A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.",
  "YearOfRelease": 1997
}
```


### Setup

In [8]:
import os
import json
import chromadb
from chromadb.utils import embedding_functions

In [9]:
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
# CHROMA_OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
# Local Chroma instance does not require API key; use OpenAI API key for OpenAI embeddings with Chroma
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")

### VectorDB Instance

In [10]:
# Instantiate Chroma persistent client
chroma_client = chromadb.PersistentClient(path="chromadb")

### Collection

In [11]:
embedding_fn = embedding_functions.OpenAIEmbeddingFunction(api_key=OPENAI_API_KEY)

In [12]:
collection_name = "udaplay"

collection = chroma_client.get_or_create_collection(
   name=collection_name,
   embedding_function=embedding_fn
)

### Add documents

In [13]:
# Make sure you have a directory "project/starter/games"
data_dir = "games"

for file_name in sorted(os.listdir(data_dir)):
    if not file_name.endswith(".json"):
        continue

    file_path = os.path.join(data_dir, file_name)
    with open(file_path, "r", encoding="utf-8") as f:
        game = json.load(f)

    # You can change what text you want to index
    content = f"[{game['Platform']}] {game['Name']} ({game['YearOfRelease']}) - {game['Description']}"

    # Use file name (like 001) as ID
    doc_id = os.path.splitext(file_name)[0]

    collection.add(
        ids=[doc_id],
        documents=[content],
        metadatas=[game]
    )

## Semantic Search

In [16]:
query = "When was Gran Turismo 5 released?"

results = collection.query(
    query_texts=[query],
    n_results=3,
    include=['documents', 'metadatas']
    )

results

{'ids': [['003', '001', '002']],
 'embeddings': None,
 'documents': [['[PlayStation 3] Gran Turismo 5 (2010) - A comprehensive racing simulator featuring a vast selection of vehicles and tracks, with realistic driving physics.',
   '[PlayStation 1] Gran Turismo (1997) - A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.',
   "[PlayStation 2] Grand Theft Auto: San Andreas (2004) - An expansive open-world game set in the fictional state of San Andreas, following the story of Carl 'CJ' Johnson."]],
 'uris': None,
 'included': ['documents', 'metadatas'],
 'data': None,
 'metadatas': [[{'Name': 'Gran Turismo 5',
    'Genre': 'Racing',
    'YearOfRelease': 2010,
    'Publisher': 'Sony Computer Entertainment',
    'Description': 'A comprehensive racing simulator featuring a vast selection of vehicles and tracks, with realistic driving physics.',
    'Platform': 'PlayStation 3'},
   {'Name': 'Gran Turismo',
    'Platform': 'PlayStation